# Ethiopia Mapping Section

This Python Jupyter file is to calculate and build out the requirements for the Ethiopian design. It could be possible to do this in Excel as well; but this way we have an ability to reference and redesign as per the changing requirements.

First Step is to collect the information; I have a Github account under my username (johnmeye) which i will reference from the file so that anyone who uses Conda/Jupyter will be able to get the files. For any challenges reach out to me on teams or by email (johnmeye@cisco.com)

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#For the Map Plotting
import plotly.express as px

#Fo the Widgets
import ipywidgets as widgets #Importing Widgets to allow for the changing of variables on the fly as questions are asked.

## Geographical Data

This file below is from the Ethiopian files available from the database on the following site:
https://data.humdata.org/dataset/ethiopia-population-data-_-admin-level-0-3

This site has both topography and the Level 0-3 admin data on the population levels per county/province. Vodacom only provided at Admin 1 (Provincial level) for this RFQ; but we are able to go more in depth to try and work out if there is any additional information we can use to strengthen our position.

The following section will pull that information from my Github; so that you don't have to fetch it yourself. Then load it as a JSON file into the DB, which you can see are Polygon type files with GPS coordinates which mark out the different layers/levels in the country. 

In [ ]:
#Pull the Data I stored in my Github account for the analysis.

with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm1_csa_bofed_20190827.json') as response:
    counties1 = json.load(response)
    
with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm2_csa_bofed_20190827.json') as response:
    counties2 = json.load(response)
    
with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm3_csa_bofed_20190827.json') as response:
    counties3 = json.load(response)

#Feature Data is available in the JSON files but its easier to manage from a tableset so i have pulled this below as well.
    
Boundaries_Data1 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/eth_adminboundaries_tabulardata.xlsx",
                  sheet_name='Admin1')

Boundaries_Data3 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/eth_adminboundaries_tabulardata.xlsx",
                    sheet_name='Admin3')


In [ ]:
counties3["features"][0]['properties'] #Just a sample on how to pull out specific information from the Counties json Files.

## Admin Level Data

As mentioned above; there is both Geo and Admin data; this information matches the information against some paramater; since the file is nicely structured according to standards we will stick to the humanitarian markings. 

Below i read the information from different levels into the variables for Admin1-3 so that we are able to use them to draw choropleth maps of the country. 

Once read into memory; it is possible to find matches against the specific parameters in both the GeoJSON and the Admin files. So i run a few sample commands to view what the data looks like. 



In [ ]:
Admin1 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin1')

Admin2 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin2')

Admin3 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin3')

In [ ]:
Admin1.info()

# Plotting the information on a Choropleth map

This information that we have gathered above needs to be represented in order to accurately help. 

Vodacom has provided Admin 1 information so we will plot against the Admin1 codes in the information. 

I have made it all at different levels so that some code can run quicker if need be.

In [ ]:
fig = px.choropleth_mapbox(Admin1, 
                           geojson=counties1, 
                           locations='admin1Pcode', featureidkey="properties.ADM1_PCODE",
                           color='Total Population',
                           color_continuous_scale="portland",
                           range_color=(500000, 30000000),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 9, "lon": 39},
                           opacity=0.5,
                           hover_name='admin1Name_en',
                           labels={
                            'Population_Density':'Population Density',
                            'Total_Population':'Total Population',
                            'admin1Name_en':'Province',
                            'Area_Km':'Square Kilometer'
                            },
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()


# Manipulating the data to find the sites requirements

Vodacom has provided the Ethiopia site numbers, and the expected by year and by type, although the Vodacom breaks it down by height and rooftop; this might not be necessary from our point of view and should not impact the way we calculate this. 

For this we will need to figure out how to define rural/urban and so forth.

In [ ]:
Sites = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/TX%20BoQ%20v3-Python.xlsx",
                   sheet_name='Site_Numbers')

Sites.rename(columns = {'Location':'admin1Name_en'}, inplace = True)
Site_Year = Sites.groupby(['admin1Name_en']).sum()
TotalSites = pd.merge(Site_Year, Boundaries_Data1[['admin1Name_en', 'Shape_Area']],how='left', on=['admin1Name_en']) #Site combined with the geodata. Sizing found early.
TotalSites.rename(columns = {'Shape_Area':'Shape_Area_Admin1'}, inplace = True)

print(TotalSites)



In [ ]:
TotalSites.sum(axis = 0, skipna = True)

In [ ]:
SiteTable = pd.merge(Admin3, TotalSites, how='outer', on=['admin1Name_en']) #Site by year

In [ ]:
SiteTable.replace(to_replace=0, value=np.nan, inplace=True)

In [ ]:
Fulltable = pd.merge(SiteTable, Boundaries_Data3[['admin3Pcode', 'Shape_Area']],how='left', on=['admin3Pcode']) #Site combined with the geodata. Sizing found early.

Fulltable.sample()

In [ ]:
Fulltable['Shape_Area'] = Fulltable['Shape_Area'].fillna(0)

In [ ]:
is_NaN = pd.isnull(Fulltable['Shape_Area'])

In [ ]:
Fulltable[is_NaN]  

### TODO: 
Plan is to create a dynamic image with the section below so that you can switch between the years and view what is happening.

In [ ]:
#fig = px.choropleth_mapbox(Fulltable, 
#                           geojson=counties1, 
#                           locations='admin1Name_en', featureidkey="properties.ADM1_EN",
#                           color='Year1',
#                           color_continuous_scale="portland",
#                           range_color=(0, 350),
#                           mapbox_style="carto-positron",
#                           zoom=3, center = {"lat": 9, "lon": 39},
#                           opacity=0.5,
#                           
#                          )
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()

# Calculating population density

The files provide a value for the size of the land and the overall land coverage. 1,104,300 square Km is the total size of the land in question. The following section shows the calculations used to determine the density for the various provinces. Which we are then able to use for the density and Rural/Urban calculations.

In [ ]:
Ethiopia_Area = 1104300
Ethiopia_Shape_Area = Fulltable['Shape_Area'].sum()
Unit_Area = Ethiopia_Area / Ethiopia_Shape_Area
Fulltable['Area_Km'] = Fulltable['Shape_Area'].apply(lambda x: x*Unit_Area)
Fulltable.rename(columns = {'Total Population':'Total_Population'}, inplace = True)
Fulltable.info()

In [ ]:
def POPDENSITY(Population,Area):
    try:
        Density=Population/Area
        return Density
    except ZeroDivisionError:
        return 0

Fulltable['Population_Density'] = Fulltable.apply(lambda x: POPDENSITY(x.Total_Population, x.Area_Km), axis=1)

In [ ]:
Fulltable['Population_Density'].sample(10)

In [ ]:
plt.figure(figsize=(15,10))
plt.tight_layout()
sns.boxplot(x="admin1Name_en", y="Total_Population", data=Fulltable,palette='rainbow')
plt.show()

In [ ]:
def CHOROMAP(DF,JSON,LOCATION,FEATUREKEY,COLOR,RANGE_L,RANGE_U, TITLE):
    Figure = px.choropleth_mapbox(DF, 
                        geojson=JSON, 
                        locations=LOCATION, featureidkey=FEATUREKEY,
                        color=COLOR,
                        color_continuous_scale="portland",
                        range_color=(RANGE_L, RANGE_U),
                        mapbox_style="carto-positron",
                        zoom=3, center = {"lat": 9, "lon": 39},
                        opacity=0.5,
                        hover_name='admin3Name_en',
                        hover_data=["admin1Name_en", "Total_Population", "Area_Km"],
                        labels={
                            'Population_Density':'Population Density',
                            'Total_Population':'Total Population',
                            'admin1Name_en':'Province',
                            'Area_Km':'Square Kilometer'
                        },
                        title = TITLE
                        )
    
    Figure.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return Figure

JSON_ON     = counties3
Location_ON = 'admin3Pcode'
Feature_ON  = 'properties.ADM3_PCODE'

Color_ON    = 'Total_Population'

Pop_Map = CHOROMAP(Fulltable,JSON_ON, Location_ON, Feature_ON, Color_ON, 25000, 300000,'Population')

Color_ON    = 'Population_Density'

Dens_Map = CHOROMAP(Fulltable,JSON_ON, Location_ON, Feature_ON, Color_ON, 50, 500,'Population Density')


In [ ]:
Pop_Map.show()
Dens_Map.show()

# Figuring out the site types

This is the step just prior to working out how many of each kind of router we will need. Now that we know about the different population densities; we can catagories them into the different catagories that they have provided for us. 

Catagories are: 
1. Dense-Urban
2. Urban
3. Sub-Urban
3. Rural
4. Deep-Rural

The result of this catagorising would be that the system would then know how the access type needs to be designed. The following image shows how they expect it to be.

![Microwave Connection](SiteConnection.png)

The breakdown would require then that there are specific rules in place for how many towers per catagory before we have a fibre connection. 

The Ethiopian Access Microwave network needs to ensure the highest level of stability. It is also important that network remains congestion free during failures. Therefore, the following guidelines should be adhered to:

1. Maximum of 4 Microwave Hops in a chain, 3 Hops in a chain according to capacities of the Vodafone calculator. The 4th hop (closest to the traditional BSC or aggregation backhaul point) needs to be protected. Plan for double the capacity required. These N+0 links must be used.
2. 5 Microwave sites in a star configuration (1st hop closest to the traditional BSC or aggregation point needs to be protected, planned for double capacity)
3. 4 Microwave Hops in chain protected by a full Microwave Ring 
4. 4 Microwave Hops in chain protected by a Mixed MW and Fibre Ring
5. Mesh topology with multiple aggregation points

The following image shows the planned layout of the network as per the Ethiotel backbone.

![Core Network](Core_Network.jpg)



In [ ]:
plt.figure(figsize=(15,10))
plt.tight_layout()
sns.boxplot(x="admin1Name_en", y="Population_Density", data=Fulltable,palette='rainbow')


## Understanding the population density

The above figure shows that there is a some extremely dense portions of the country, Addis, Dire, Tigray, with some outlier towns in many of the other regions.

I suspect though that the towns might be showing as overly dense because rural people register at the towns and push up the numbers artificially. 

But we have two pieces of information which will help to decide what we do.

1. The population for a specific area
2. The calculated population density for an area which is calculated from the total country size and the region size.

Using these two together we should be able to place the regions into specific buckets. 

1. Dense Urban = Population Denser then 1000 people per km^2
2. Urban = Population Denser then 500 people but less then 1000 people per km^2
3. Sub-Urban = Population density between 200 and 500 people per km^2
4. Rural = Population density between 100 and 200 people per km^2
5. Deep-Rural = Population density less then 100 people per km^2

### Todo: Make this a dropdown so that we can select the range of each one to play around with it and figure out what is best.

In [ ]:
DU_Thresh = 1000
U_Thresh = 500
SU_Thresh = 200
R_Thresh = 100
DR_Thresh = 0

def AREA_TYPE(Population_Density):
    if Population_Density >= DU_Thresh:
        return 'Dense Urban'
    elif U_Thresh <= Population_Density < DU_Thresh:
        return 'Urban'
    elif SU_Thresh <= Population_Density < U_Thresh:
        return 'Sub-Urban'
    elif R_Thresh <= Population_Density < SU_Thresh:
        return 'Rural'
    else:
        return 'Deep Rural'

Fulltable['Area_Type'] = Fulltable.apply(lambda x: AREA_TYPE(x.Population_Density), axis=1)

In [ ]:
Fulltable['Area_Type'].value_counts()

In [ ]:
Fulltable[Fulltable['admin1Name_en']=='Addis Ababa']['Area_Type'].value_counts()

# Using Area_Type to define what Routers will be needed

## Information available

The Choropleth map can not be subdivided into regions and sites. We have the information on the following to help make this decision

1. Population in an area
2. Size of an area
3. Type of area
4. Number of sites planned by vodacom for the area

## Site counts

Vodacom has given the number of sites it plans on a provincial level. But as can already be seen by the images above, the sites are extremely large and should be broken down further. Their methods they used was to break up the area types per year, Rural/Suburban etc... The question now comes in on how to approach this division. 

## Site connectivity
With this information we should be able to make the decision of how to split up the sites and how to connect them.

The breakdown is as follows:

1. For Rural and Deep Rural; the sites will all be MW linking back to the backhaul via MW sites. 
2. For Sub-Urban we will be able to have a mix of MW and Fibre, back to the nearest Urban location. Typically less then 80km away. 
3. For the Urbun and Dense-Urbun, we will be able to have fibre exclusively according to Vodacom planning.


In [ ]:
for i in Fulltable.admin1Name_en.unique():
    
    Area_Breakdown = Fulltable[Fulltable['admin1Name_en']== i]['Area_Type'].value_counts()
    print(i)
    print(Area_Breakdown)


In [ ]:
dfDU = Fulltable[Fulltable['Area_Type']=='Dense Urban']
dfU = Fulltable[Fulltable['Area_Type']=='Urban']
dfSU = Fulltable[Fulltable['Area_Type']=='Sub-Urban']
dfR = Fulltable[Fulltable['Area_Type']=='Rural']
dfDR = Fulltable[Fulltable['Area_Type']=='Deep Rural']

In [ ]:
def CHOROMAP(DF,JSON,LOCATION,FEATUREKEY,COLOR,RANGE_L,RANGE_U, TITLE):
    Figure = px.choropleth_mapbox(DF, 
                        geojson=JSON, 
                        locations=LOCATION, featureidkey=FEATUREKEY,
                        color=COLOR,
                        color_continuous_scale="portland",
                        range_color=(RANGE_L, RANGE_U),
                        mapbox_style="carto-positron",
                        zoom=3, center = {"lat": 9, "lon": 39},
                        opacity=0.5,
                        hover_name='admin3Name_en',
                        hover_data=["admin1Name_en", "Total_Population", "Area_Km"],
                        labels={
                            'Population_Density':'Population Density',
                            'Total_Population':'Total Population',
                            'admin1Name_en':'Province',
                            'Area_Km':'Square Kilometer'
                                },
                        title = TITLE
                          )
    
    Figure.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return Figure

JSON_ON     = counties3
Location_ON = 'admin3Pcode'
Feature_ON  = 'properties.ADM3_PCODE'
Color_ON    = 'Population_Density'

DU_Map = CHOROMAP(dfDU,JSON_ON, Location_ON, Feature_ON, Color_ON, DU_Thresh, 5000,'Dense Urban')
U_Map  = CHOROMAP(dfU,JSON_ON, Location_ON, Feature_ON, Color_ON, U_Thresh, DU_Thresh, 'Urban')
SU_Map = CHOROMAP(dfSU,JSON_ON, Location_ON, Feature_ON, Color_ON, SU_Thresh, U_Thresh, 'Sub Urban')
R_Map  = CHOROMAP(dfR,JSON_ON, Location_ON, Feature_ON, Color_ON, R_Thresh, SU_Thresh, 'Rural')
DR_Map = CHOROMAP(dfDR,JSON_ON, Location_ON, Feature_ON, Color_ON, DR_Thresh, R_Thresh, 'Deep Rural')



In [ ]:
DU_Map.show()
DR_Map.show()